In [2]:
import os
os.listdir()

['.ipynb_checkpoints',
 'Credit Card Fraud Detection.ipynb',
 'Credit card Fraud Detection.py',
 'fraudTest.csv',
 'fraudTrain.csv',
 'fraud_detection_model.pkl',
 'imdb_top_1000.csv',
 'Movie Genre Classification.ipynb',
 'Movie Genre Classification.py']

In [12]:
import pandas  as pd 
import warnings
warnings.filterwarnings("ignore")
data=pd.read_csv("description.txt")
data

,Train data:
0,ID ::: TITLE ::: GENRE ::: DESCRIPTION
1,ID ::: TITLE ::: GENRE ::: DESCRIPTION
2,ID ::: TITLE ::: GENRE ::: DESCRIPTION
3,ID ::: TITLE ::: GENRE ::: DESCRIPTION
4,Test data:
5,ID ::: TITLE ::: DESCRIPTION
6,ID ::: TITLE ::: DESCRIPTION
7,ID ::: TITLE ::: DESCRIPTION
8,ID ::: TITLE ::: DESCRIPTION
9,Source:


In [7]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data= f.readlines()
    data= [line.strip().split(':::') for line in data]
    return data

In [11]:
train_data = load_data("train_data.txt")
train_df = pd.DataFrame(train_data, columns=['ID', 'Title', 'Genre', 'Description'])

test_data = load_data("test_data.txt")
test_df = pd.DataFrame(test_data, columns=['ID', 'Title', 'Description'])

test_solution= load_data('test_data_solution.txt')
test_solution_df = pd.DataFrame(test_solution, columns=['ID', 'Title', 'Genre', 'Description'])


In [13]:
print("\nTrain Data:")
train_df



Train Data:


,ID,Title,Genre,Description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on ...
54210,54211,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The sist...
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about g..."
54212,54213,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and hav...


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000)

X_train_tfidf = vectorizer.fit_transform(train_df["Description"])
X_test_tfidf= vectorizer.transform(test_df["Description"])

print(f"Training data shape:{X_train_tfidf.shape}")

print(f"Test data shape: {X_test_tfidf.shape}")

Training data shape:(54214, 10000)
Test data shape: (54200, 10000)


In [17]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_train = label_encoder.fit_transform(train_df['Genre'])
print(f"Unique genres in the training data:{label_encoder.classes_}")


Unique genres in the training data:[' action ' ' adult ' ' adventure ' ' animation ' ' biography ' ' comedy '
 ' crime ' ' documentary ' ' drama ' ' family ' ' fantasy ' ' game-show '
 ' history ' ' horror ' ' music ' ' musical ' ' mystery ' ' news '
 ' reality-tv ' ' romance ' ' sci-fi ' ' short ' ' sport ' ' talk-show '
 ' thriller ' ' war ' ' western ']


In [18]:
from sklearn.linear_model import LogisticRegression
lr_model= LogisticRegression(max_iter=200)
lr_model.fit(X_train_tfidf, y_train)

y_pred = lr_model.predict(X_test_tfidf)
predicted_genres= label_encoder.inverse_transform(y_pred)

test_df['Predicted_Genre'] = predicted_genres
test_df[['Title', 'Predicted_Genre']]

,Title,Predicted_Genre
0,Edgar's Lunch (1998),drama
1,La guerra de papá (1977),drama
2,Off the Beaten Track (2010),documentary
3,Meu Amigo Hindu (2015),drama
4,Er nu zhai (1955),drama
...,...,...
54195,"""Tales of Light & Dark"" (2013)",drama
54196,Der letzte Mohikaner (1965),drama
54197,Oliver Twink (2007),comedy
54198,Slipstream (1973),drama


In [19]:
test_df['Predicted_Genre']= predicted_genres
merged_df=pd.merge(test_solution_df[['ID','Genre']],test_df[['ID','Predicted_Genre']],on='ID')
merged_df

,ID,Genre,Predicted_Genre
0,1,thriller,drama
1,2,comedy,drama
2,3,documentary,documentary
3,4,drama,drama
4,5,drama,drama
...,...,...,...
54195,54196,horror,drama
54196,54197,western,drama
54197,54198,adult,comedy
54198,54199,drama,drama


In [21]:
from sklearn.metrics import accuracy_score, classification_report

accuracy= accuracy_score(merged_df ['Genre'], merged_df[ 'Predicted_Genre'])
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(merged_df['Genre'], merged_df['Predicted_Genre']))


Accuracy: 0.5946

Classification Report:
               precision    recall  f1-score   support

      action        0.50      0.30      0.37      1314
       adult        0.65      0.24      0.35       590
   adventure        0.67      0.16      0.25       775
   animation        0.59      0.04      0.08       498
   biography        0.00      0.00      0.00       264
      comedy        0.54      0.60      0.57      7446
       crime        0.41      0.03      0.06       505
 documentary        0.68      0.87      0.76     13096
       drama        0.55      0.79      0.65     13612
      family        0.50      0.08      0.14       783
     fantasy        0.65      0.03      0.06       322
   game-show        0.90      0.49      0.64       193
     history        0.00      0.00      0.00       243
      horror        0.66      0.57      0.61      2204
       music        0.67      0.46      0.55       731
     musical        0.44      0.01      0.03       276
     mystery        0.2

In [23]:
from sklearn.naive_bayes import MultinomialNB
nb_model=MultinomialNB()
nb_model.fit(X_train_tfidf,y_train)

MultinomialNB()

In [24]:
y_pred_nb=nb_model.predict(X_test_tfidf)
predicted_genres_nb= label_encoder.inverse_transform(y_pred_nb)
test_df['Predicted_Genre_NB'] = predicted_genres_nb
merged_df_nb = pd.merge(test_solution_df, test_df[['ID', 'Predicted_Genre_NB']], on='ID')

In [25]:
from sklearn.metrics import accuracy_score, classification_report

accuracy_nb = accuracy_score(merged_df_nb['Genre'], merged_df_nb[ 'Predicted_Genre_NB'])
print(f"Naive Bayes Accuracy: {accuracy_nb}")

print("Naive Bayes Classification Report:")
print(classification_report(merged_df_nb['Genre'], merged_df_nb[ 'Predicted_Genre_NB'],target_names=label_encoder.classes_))

Naive Bayes Accuracy: 0.5092435424354244
Naive Bayes Classification Report:
               precision    recall  f1-score   support

      action        0.56      0.03      0.06      1314
       adult        0.46      0.02      0.04       590
   adventure        0.77      0.04      0.08       775
   animation        0.00      0.00      0.00       498
   biography        0.00      0.00      0.00       264
      comedy        0.53      0.40      0.46      7446
       crime        0.00      0.00      0.00       505
 documentary        0.56      0.89      0.69     13096
       drama        0.44      0.84      0.58     13612
      family        0.00      0.00      0.00       783
     fantasy        0.00      0.00      0.00       322
   game-show        1.00      0.02      0.04       193
     history        0.00      0.00      0.00       243
      horror        0.77      0.23      0.35      2204
       music        0.89      0.02      0.05       731
     musical        0.00      0.00      0.0

In [26]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [ ]:
y_pred_svm = svm_model.predict(X_test_tfidf)
predicted_genres_svm=label_encoder.inverse_transform(y_pred_svm)
test_df['Predicted_Genre_SVM'] = predicted_genres_svm
merged_df_svm = pd.merge(test_solution_df, test_df[['ID', 'Predicted_Genre_SVM']], on='ID')

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

accuracy_svm = accuracy_score(merged_df_svm[ "Genre"], merged_df_svm['Predicted_Genre_SVM'])
print(f"SVM Accuracy: {accuracy_svm}")

print("SVM Classification Report:")
print(classification_report(merged_df_svm ['Genre'], merged_df_svm['Predicted_Genre_SVM'], target_names=label_encoder.classes_))

In [ ]:
zoner_Description = [
    'Explosive fight scenes in the city streets',
    'A haunted mansion that traps its visitors',
    'A brave adventurer in search of lost treasure', 
    'A forbidden romance in the 1920s',
    'A daring rescue mission with a love interest'
]

#Step 1: Vectorize the new test data using the same vectorizer

test_data_tfidf = vectorizer.transform(zoner_Description) 

#Step 2: Predict genres using each model

y_pred_lr = lr_model.predict(test_data_tfidf) 
predicted_genres_lr=label_encoder.inverse_transform(y_pred_1r) # Inverse transform to get genre names ad_genres

y_pred_nb = nb_model.predict(test_data_tfidf) # Predict using Naive Bayes
predicted_genres_nb = label_encoder.inverse_transform(y_pred_nb) # Inverse transform to get genre names

y_pred_svm = svm_model.predict(test_data_tfidf) # Predict using SVM
predicted_genres_svm=label_encoder.inverse_transform(y_pred_svm) # Inverse transform to get genre names

#step 3:
print("Predicted Genres using Logistic Regression: ", predicted_genres_lr)
print("Predicted Genres using Naive Bayes:", predicted_genres_nb)
print("Predicted Genres using SVM:", predicted_genres_svm)
print()
for i, message in enumerate (zoner_Description):
    print(f"Story: {message}")
    print(f"Status: \tNaive Bayes Prediction :{predicted_genres_nb[i]}")
    print(f"\t\tLogistic Regression Prediction: {predicted_genres_lr[i]}")

print(f"\t\tSVM Prediction:{predicted_genres_svm[i]}")

print("="*100) # Separates each message